# Prueba de evaluación 2

Realizado por Araceli Macía Barrado

## Parte 3

El sitio Web NOOA del gobierno de EEUU proporciona datasets de datos climáticos a través de esta página Web:
http://www.ncdc.noaa.gov/cdo-web/datasets

    
Entre ellos tenemos los datasets “Quality Controlled Local Climatological Data (QCLCD)” que se describen aquí: http://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based- datasets/quality-controlled-local-climatological-data-qclcd

Entre los datos que se encuentran en los datasets QCLCD están las precipitaciones por años y estaciones. Por ejemplo, podemos descargar los datasets de aquí:

http://www.ncdc.noaa.gov/orders/qclcd/

Y encontraremos ficheros con datos de precipitaciones como este:
Wban Number, YearMonthDay, Time, Hourly Precip 03013,19960701,0053,0
03013,19960701,0153,0
03013,19960701,0253,0
03013,19960701,0353,0 03013,19960701,0453,0 ...

Se pide tomar datos de varios años (queda a la elección del estudiante) de este conjunto de datasets para las precipitaciones y obtener los siguientes resúmenes:

- Día en que ha habido más precipitaciones.
- Año en que ha habido más precipitaciones (obteniendo la media de cada año) 


Se pide realizar el análisis en dos versiones:
- Una utilizando DataFrames y los ficheros de texto que se decargan directamente.
- Una segunda con un paso previo en el que se guardan los datos en un fichero HDF5 (que debe contener los metadatos descriptivos necesarios). Queda a la decisión del estudiante cómo organizar los datos en el fichero.

Y se pide comparar:
- El tamaño en disco que ocupan los datos en cada una de las versiones. - El tiempo comparado de ejecución de los resúmenes anteriores.

Opcional: finalmente, se platea el almacenar en el fichero HDF5 los resúmenes mismos obtenidos y comparar el tiempo de recuperación de esos datos del fichero con el tiempo tardado en calcularlo.

-----

Para la realizacion de la practica me he bajado los ficheros del mes de Abril de los año 2012 al 2016.
Los tengo en la carpeta datos.


###  Una utilizando DataFrames y los ficheros de texto que se decargan directamente.


In [1]:
import numpy as np
import pandas as pd
import glob
import cProfile

def str2f(x):
    
    x1=str(x)[0:4]
    x1= x1.strip(" ")                       
    if x1=='M' or x1.strip()=='T' or x1=="" or x1=="TE":       
        return float("0")              
    else:  
        return float(x1)

def sstring(x):
    return str(x)

ficheros= glob.glob("datos/*.txt") #almaceno todos los nombres de los ficheros txt que tengo en el directorio.

In [2]:

dataTotal=pd.DataFrame()
filas=0
for fic in ficheros:
    data1= pd.read_csv(fic,index_col=None, header=0, usecols=["YearMonthDay", "Precipitation"],\
                       converters={'Precipitation':str2f, 'YearMonthDay':sstring}) 
    filas=filas+ data1.shape[0]
    dataTotal=dataTotal.append(data1,ignore_index=True)





In [3]:
print "numero de filas" , filas  #reviso que tengo todas las filas y que el indice ha quedado bien
print dataTotal.shape
print dataTotal.tail(1)

numero de filas 7643520
(7643520, 2)
        YearMonthDay  Precipitation
7643519     20160430            0.0


In [4]:
#para poder comparar el tamaño, voy a volcar el dataframe resultante a un csv
%rm lluvias.csv
dataTotal.to_csv('lluvia.csv')


rm: lluvias.csv: No such file or directory


Ahora voy a revisar los datos que se han cargado en YearMonthDay y PrecipTotal que son los valores que me interesan.

In [5]:
dataTotal['YearMonthDay'].unique()

array(['20120401', '20120402', '20120403', '20120404', '20120405',
       '20120406', '20120407', '20120408', '20120409', '20120410',
       '20120411', '20120412', '20120413', '20120414', '20120415',
       '20120416', '20120417', '20120418', '20120419', '20120420',
       '20120421', '20120422', '20120423', '20120424', '20120425',
       '20120426', '20120427', '20120428', '20120429', '20120430',
       '20130401', '20130402', '20130403', '20130404', '20130405',
       '20130406', '20130407', '20130408', '20130409', '20130410',
       '20130411', '20130412', '20130413', '20130414', '20130415',
       '20130416', '20130417', '20130418', '20130419', '20130420',
       '20130421', '20130422', '20130423', '20130424', '20130425',
       '20130426', '20130427', '20130428', '20130429', '20130430',
       '20140401', '20140402', '20140403', '20140404', '20140405',
       '20140406', '20140407', '20140408', '20140409', '20140410',
       '20140411', '20140412', '20140413', '20140414', '201404

In [6]:
dataTotal["anio"]=dataTotal["YearMonthDay"].str[0:4]
print dataTotal.anio.unique()

['2012' '2013' '2014' '2015' '2016']


De los datos que hay, no voy a considerar los que tienen que lluvia ha sido 0, porque desvirtuaria los calculos de las precipitaciones.

In [7]:
%time dataTratar=dataTotal.loc[dataTotal["Precipitation"]>0]
print dataTratar.shape  #el numero de registros a tratar ha bajado considerablemente.


CPU times: user 419 ms, sys: 117 ms, total: 536 ms
Wall time: 571 ms
(367919, 3)


### - Dia en que ha habido mas precipitaciones

In [8]:
%time  dataTratar[dataTratar["Precipitation"]==dataTratar["Precipitation"].max()]

CPU times: user 4.12 ms, sys: 1.12 ms, total: 5.24 ms
Wall time: 3.77 ms


,YearMonthDay,Precipitation,anio
7002973,20160411,11.5,2016


> **el dia que hubo mas precipitaciones fue el 11 de abril de 2016.**

### - Año en que ha habido mas precipitaciones obteniendo la media por año

In [9]:
anioMax=dataTratar.groupby("anio")
anioMax["Precipitation"].mean()

anio
2012    0.064002
2013    0.064294
2014    0.067134
2015    0.072001
2016    0.070388
Name: Precipitation, dtype: float64

In [10]:
anioMax["Precipitation"].mean().argmax()

'2015'

> **El año 2015 fue el que tuvo el mes de abril mas lluvioso.**

------------

###  Con paso previo, guardando los datos en un fichero HDF5


In [11]:
%rm lluvia.hdf5
#store.close()

In [12]:
import os
import tables as tb
import numpy as np
import pandas as pd
import glob

import numpy as np
import pandas as pd
import glob
import cProfile

def str2f(x):
    
    x1=str(x)[0:4]
    x1= x1.strip(" ")                       
    if x1=='M' or x1.strip()=='T' or x1=="" or x1=="TE":       
        return float("0")              
    else:  
        return float(x1)

def sstring(x):
    return str(x)

ficheros= glob.glob("datos/*.txt") #almaceno todos los nombres de los ficheros txt que tengo en el directorio.

lines = 50000 #se suben en chunk de 5000 lineas.
store = pd.HDFStore('lluvia.hdf5',mode='w')

#En el fichero h5 almaceno los datos por año.
for fic in ficheros:  
    for chunk in pd.read_csv(fic, chunksize=lines, \
                         index_col=None, header=0, usecols=["YearMonthDay", "Precipitation"],converters={'Precipitation':str2f, 'YearMonthDay':sstring}): 

        nom="DTTotales"
        store.append(nom,chunk)
        store.get_storer(nom).attrs.descripcion="Datos de Abril de 2012,2013,2014,2015 y 2016"  #inserto atributos
        store.get_storer(nom).attrs.fechaDatos="20161014"

print store.select("DTTotales").shape
store

(7643520, 2)


<class 'pandas.io.pytables.HDFStore'>
File path: lluvia.hdf5
/DTTotales            frame_table  (typ->appendable,nrows->7643520,ncols->2,indexers->[index])

el numero de filas 7643520 coincide. Se ha hecho la carga correctamente.

### - Dia en que ha habido mas precipitaciones

In [14]:
%time  store.select("DTTotales")[store.select("DTTotales")["Precipitation"]==store.select("DTTotales")["Precipitation"].max()]


CPU times: user 3.4 s, sys: 2.51 s, total: 5.91 s
Wall time: 7.27 s


,YearMonthDay,Precipitation
28413,20160411,11.5


> El dia que hubo mas precipitaciones, el 11 de Abril del 2016

**Comparando el tiempo de ejecucion con la instruccion arriba ejecutada:**

%time  dataTratar[dataTratar["Precipitation"]==dataTratar["Precipitation"].max()]

**se aprecia que el tiempo en realizar esta ejecucion ha sido <font color="red"> mayor </font>, puesto que en esta lectura los datos estan en disco.**


### - Año en que ha habido mas precipitaciones obteniendo la media por año

In [15]:
#Para hacer esto, tengo que pasar los datos un dataframe en memoria, si ejecuto las instrucciones directamente 
#en el store, tarda una eternidad..

#store.select("DTTotales").loc[store.select("DTTotales")["Precipitation"]>0].groupby(store.select("DTTotales")["YearMonthDay"].str[0:4])

%time datosAtratar=store.select("DTTotales").loc[store.select("DTTotales")["Precipitation"]>0] 



CPU times: user 1.84 s, sys: 1.3 s, total: 3.14 s
Wall time: 3.61 s



**Comparando el tiempo de ejecucion con la instruccion arriba ejecutada:**

%time dataTratar=dataTotal.loc[dataTotal["Precipitation"]>0]

**se aprecia que el tiempo en realizar esta ejecucion ha sido <font color="red"> mucho mayor </font>, puesto que en esta lectura los datos estan en disco.**

In [16]:
datosAtratar["anio"]=datosAtratar["YearMonthDay"].str[0:4]
print datosAtratar.anio.unique()

['2012' '2013' '2014' '2015' '2016']


In [17]:
anioMax=datosAtratar.groupby("anio")
anioMax["Precipitation"].mean()

anio
2012    0.064002
2013    0.064294
2014    0.067134
2015    0.072001
2016    0.070388
Name: Precipitation, dtype: float64

In [18]:
anioMax["Precipitation"].mean().argmax()

'2015'

> **El año 2015 fue el que tuvo el mes de abril mas lluvioso.**

In [19]:
store.close()


### Comparar el tamaño en disco que ocupan los datos.

In [20]:
%ls -ltr lluvia*.*

-rw-r--r--  1 aramaciabarrado  staff  159759948 16 oct 17:35 lluvia.csv
-rw-r--r--  1 aramaciabarrado  staff  192708058 16 oct 17:37 lluvia.hdf5


el fichero h5df es mayor que el csv teniendo los dos el mismo  numero de filas y columnas.
He estado revisando documentacion por internet, y tras ejecutar esta linea de comando:

 h5ls -vr /Users/aramaciabarrado/Documents/*.h5df 
 
me devuelve una serie de datos,  y al final acabo diciendo esto:

in h5tools_dump_mem(): H5Sis_simple failed

la compresion falla..


------------
### Revisar los tiempos de ejecucion

#### A) Comparar tiempo de lectura entre un fichero csv y un fichero h5df.

In [21]:
import StringIO
import pstats

ficheros= glob.glob("datos/*.txt") #almaceno todos los nombres de los ficheros txt que tengo en el directorio.
dataTotal=pd.DataFrame()
filas=0

%rm Times.log
pr = cProfile.Profile()
pr.enable()
#voy a leer el fichero lluvia.csv que es donde habia volcado el dataframe con todos los datos.
data1 = pd.read_csv("lluvia.csv", index_col=None, header=0, usecols=["YearMonthDay", "Precipitation"])

pr.disable()
pr.dump_stats("Times.log")
s = StringIO.StringIO()
sortby = 'filename'
ps = pstats.Stats("Times.log", stream=s).sort_stats(sortby)
ps.print_stats("read")
print s.getvalue()

print data1.shape

Sun Oct 16 17:38:11 2016    Times.log

         715 function calls (708 primitive calls) in 3.974 seconds

   Ordered by: file name
   List reduced from 161 to 4 due to restriction <'read'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.633    3.633 /Users/aramaciabarrado/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1211(read)
        1    0.012    0.012    3.789    3.789 /Users/aramaciabarrado/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:256(_read)
        1    0.000    0.000    3.774    3.774 /Users/aramaciabarrado/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:758(read)
        1    3.633    3.633    3.633    3.633 {method 'read' of 'pandas.parser.TextReader' objects}



(7643520, 2)


In [22]:
import h5py
from pandas import read_hdf



%rm Times.log
pr = cProfile.Profile()
pr.enable()

dataTotal=pd.DataFrame()
hdf = read_hdf('lluvia.hdf5', "DTTotales")
    
pr.disable()
pr.dump_stats("Times.log")

s = StringIO.StringIO()
sortby = 'filename'
ps = pstats.Stats("Times.log", stream=s).sort_stats(sortby)
ps.print_stats("read")
print s.getvalue()
print hdf.shape


Sun Oct 16 17:38:14 2016    Times.log

         7471 function calls (7185 primitive calls) in 1.656 seconds

   Ordered by: file name
   List reduced from 666 to 13 due to restriction <'read'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 /Users/aramaciabarrado/anaconda/lib/python2.7/site-packages/numexpr/utils.py:61(set_vml_num_threads)
        6    0.000    0.000    0.000    0.000 /Users/aramaciabarrado/anaconda/lib/python2.7/site-packages/pandas/io/pytables.py:1630(read_metadata)
        1    0.000    0.000    1.628    1.628 /Users/aramaciabarrado/anaconda/lib/python2.7/site-packages/pandas/io/pytables.py:4019(read)
        1    0.000    0.000    1.653    1.653 /Users/aramaciabarrado/anaconda/lib/python2.7/site-packages/pandas/io/pytables.py:265(read_hdf)
        6    0.000    0.000    0.000    0.000 /Users/aramaciabarrado/anaconda/lib/python2.7/site-packages/pandas/io/pytables.py:3037(read_metadata)
        1 

**El tiempo de recuperar los datos de un fichero hdf5 es significativamente mejor.**


#### B) Comparar tiempo de escritura entre un fichero csv y un fichero h5df.

In [23]:
%rm lluvia.csv

In [24]:
%rm Times.log
pr = cProfile.Profile()
pr.enable()
data1.to_csv('lluvia.csv')
pr.disable()
pr.dump_stats("Times.log")
s = StringIO.StringIO()
sortby = 'filename'
ps = pstats.Stats("Times.log", stream=s).sort_stats(sortby)
ps.print_stats("calls")
print s.getvalue()

Sun Oct 16 17:38:44 2016    Times.log

         53525540 function calls in 24.744 seconds




In [25]:
store.close()

In [26]:
%rm Times.log
%rm lluvia.hdf5
pr = cProfile.Profile()
pr.enable()

store = pd.HDFStore('lluvia.hdf5',mode='w')
store.append("DTTotales",data1)
store.close()

pr.disable()
pr.dump_stats("Times.log")
s = StringIO.StringIO()
sortby = 'filename'
ps = pstats.Stats("Times.log", stream=s).sort_stats(sortby)
ps.print_stats("calls")
print s.getvalue()


Sun Oct 16 17:38:53 2016    Times.log

         302589 function calls (302560 primitive calls) in 5.040 seconds




El tiempo de generar un fichero h5df es significativamente mejor.


Es decir, lo optimo es generar y leer datos en ficheros hf5 ( siempre que se pueda), y luego pasarlos a memoria para poder trabajar con ellos.